In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 3000 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model_original, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post3: Fast Llama patching. Transformers = 4.45.1.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 Ti SUPER. Max memory: 15.688 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu124. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


<string>:204: SyntaxWarning: invalid escape sequence '\ '
<string>:205: SyntaxWarning: invalid escape sequence '\_'
<string>:206: SyntaxWarning: invalid escape sequence '\ '
<string>:204: SyntaxWarning: invalid escape sequence '\ '
<string>:205: SyntaxWarning: invalid escape sequence '\_'
<string>:206: SyntaxWarning: invalid escape sequence '\ '


## **Model**

In [2]:
model = FastLanguageModel.get_peft_model(
    model_original,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.9.post3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## **Data Prep with multiple prompts**

In [5]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("JingyangYi/CMV_op", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/6382 [00:00<?, ? examples/s]

['malleable', 'malleable', 'malleable', 'malleable', 'malleable', 'resistant', 'malleable', 'malleable', 'resistant', 'malleable', 'malleable', 'resistant', 'resistant', 'resistant', 'malleable', 'resistant', 'resistant', 'malleable', 'malleable', 'malleable', 'malleable', 'malleable', 'malleable', 'resistant', 'resistant', 'malleable', 'resistant', 'malleable', 'malleable', 'malleable', 'resistant', 'resistant', 'malleable', 'malleable', 'malleable', 'malleable', 'malleable', 'malleable', 'resistant', 'resistant', 'resistant', 'resistant', 'resistant', 'resistant', 'resistant', 'resistant', 'resistant', 'resistant', 'malleable', 'malleable', 'resistant', 'malleable', 'malleable', 'resistant', 'resistant', 'malleable', 'resistant', 'malleable', 'malleable', 'malleable', 'resistant', 'resistant', 'resistant', 'malleable', 'malleable', 'resistant', 'resistant', 'malleable', 'malleable', 'malleable', 'resistant', 'malleable', 'malleable', 'malleable', 'malleable', 'malleable', 'malleable'

## **Training**

In [4]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [6]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 6,382 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 797
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.678300
2,2.767800
3,2.669800
4,2.752600
5,2.610400
6,2.529700
7,2.315100
8,2.467500
9,2.387200
10,2.165600


## **Save model in HuggingFace repo**

## **GGUF / llama.cpp Conversion**

## **Infer and compare**

In [8]:
from tqdm import tqdm
def infer(dataset, model, instruction):
  results = []
  for line in tqdm(dataset, desc="Inferring on test set: "):
      inputs = tokenizer(
      [
          alpaca_prompt.format(
              instruction, # instruction
              line["input"], # input
              "") # output - leave this blank for generation!
      ], return_tensors = "pt").to("cuda")
      outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
      result = tokenizer.batch_decode(outputs)[0]

      start_marker = "### Response:\n"
      end_marker = "<|end_of_text|>"
      result = result[result.find(start_marker) + len(start_marker):
                      result.find(end_marker)]

      results.append(result)
  return results

test_dataset = load_dataset("JingyangYi/CMV_op", split='test')

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
FastLanguageModel.for_inference(model_original)

instruction = test_dataset[0]["instruction"]
original_instruction = instruction + " This is a classification task, answer only with resisent or malleable."

benchmark_results = infer(test_dataset, model_original, original_instruction)
ft_results = infer(test_dataset, model, instruction)

Inferring on test set: 100%|██████████| 1344/1344 [04:04<00:00,  5.49it/s]


## **Accuracy**

In [9]:
actuals = [line["output"] for line in test_dataset]

def accuracy(results, actuals):
  correct_count = 0
  total = len(results)
  for i in range(total):
    if results[i] == actuals[i]:
      correct_count += 1
  return correct_count / total

accu_bm = accuracy(benchmark_results, actuals)
accu_ft = accuracy(ft_results, actuals)
print(f"The original model accuracy on test set is {accu_bm:.3f}")
print(f"The finetuned model accuracy on test set is {accu_ft:.3f}")

The original model accuracy on test set is 0.450
The finetuned model accuracy on test set is 0.499
